# MLFlow runs

In [1]:
import mlflow
from mlflow.tracking import MlflowClient
import itertools
import matplotlib.pyplot as plt
from mlflow.entities import ViewType

mlflow.tracking.set_tracking_uri('../mlruns')

## Functions

In [2]:
def get_runs(experiment_name, filter_string=''):
    client = MlflowClient()
    exp_ids = [client.get_experiment_by_name(experiment_name).experiment_id]
    runs = mlflow.search_runs(experiment_ids=exp_ids, filter_string=filter_string)
    return runs


def get_best_run(runs, best_metrics):
    if runs is None or runs.shape[0] == 0:
        return None
    for metric in best_metrics:
        runs = runs.loc[runs[f'metrics.{metric}'] >= runs[f'metrics.{metric}'].max()]
    # pick first in the list
    run = runs.loc[runs.index[0]]
    return run

In [3]:
def runs_to_latex(runs, threshold_method, caption, label, grouped=False):
    cols = ['tags.mlflow.runName', 'tags.model', 'tags.method', 'metrics.test_auc', 'metrics.test_tax_yield', 'metrics.test_tax_score', 'metrics.test_effort', 'metrics.test_f1-score_1', 'metrics.test_recall_1', 'metrics.test_precision_1', 'metrics.threshold']
    cols_renamed = [ 'name', 'model', 'method', 'auc', 'yield', 'value', 'effort', 'f1-score', 'recall', 'precision', 'threshold' ]
    to_replace = {
        'model': {'tree': 'DT', 'forest': 'RF', 'logit': 'LR'},
        'method': {'imbalanced': 'IM', 'oversample': 'RO', 'weighted': 'BC'}}
    df = runs.loc[:, :]
    df = df.loc[df['tags.threshold_method']==threshold_method]
    df = df.loc[df['tags.grouped']==str(grouped)]
    df = df.sort_index(ascending=False)
    df = df.loc[:, cols]
    df = df.rename(columns=dict(zip(cols, cols_renamed)))
    df = df.replace(to_replace=to_replace)
    df.loc[:, 'name'] = df.model.str.cat(df.method, sep='-')
    df = df.drop(columns=['model', 'method']).set_index(['name'])
    if threshold_method=='default':
        df = df.drop(columns=['threshold'])
    else:
        df = df.drop(columns=['auc'])
    cols_max = df.columns.drop('effort').tolist()
    s = df.style.highlight_max(subset=cols_max, axis=0, props='bfseries: ;')
    s = s.highlight_min(subset='effort', axis=0, props='bfseries: ;')
    latex_table = s.to_latex(
        column_format="lrrrrrrr", position="ht", position_float="centering",
        hrules=True, label=label, caption=caption,
        multirow_align="t", multicol_align="r")
    return latex_table

## Reports

### MaxBoxAccV3 and PxAP

In [114]:
filter_string = 'tags.dataset = "SYNTHETIC" and status = "FINISHED" and tags.architecture_type = "vanilla"'
runs = get_runs('wsol', filter_string=filter_string)
background_filter = 'b'
runs = runs.loc[runs['tags.dataset_spec'].str.contains(background_filter)]

In [115]:
cols = ['tags.mlflow.runName', 'metrics.test_MaxBoxAccV3', 'metrics.test_PxAP']

In [116]:
runs2 = runs.loc[:, cols].sort_values(by='tags.mlflow.runName')

In [118]:
print(runs2.loc[:,['tags.mlflow.runName','metrics.test_MaxBoxAccV3']])

             tags.mlflow.runName  metrics.test_MaxBoxAccV3
12   vgg16_gradcam_synthetic_d2b                  0.320833
10   vgg16_gradcam_synthetic_d3b                  0.301667
8    vgg16_gradcam_synthetic_d4b                  0.091667
14   vgg16_gradcam_synthetic_o1b                  0.745000
4   vgg16_scorecam_synthetic_d2b                  0.430833
2   vgg16_scorecam_synthetic_d3b                  0.351667
0   vgg16_scorecam_synthetic_d4b                  0.132500
6   vgg16_scorecam_synthetic_o1b                  0.705000


In [117]:
print(runs2.loc[:,['tags.mlflow.runName','metrics.test_PxAP']])

             tags.mlflow.runName  metrics.test_PxAP
12   vgg16_gradcam_synthetic_d2b           0.395897
10   vgg16_gradcam_synthetic_d3b           0.422439
8    vgg16_gradcam_synthetic_d4b           0.131227
14   vgg16_gradcam_synthetic_o1b           0.683651
4   vgg16_scorecam_synthetic_d2b           0.538016
2   vgg16_scorecam_synthetic_d3b           0.461270
0   vgg16_scorecam_synthetic_d4b           0.215631
6   vgg16_scorecam_synthetic_o1b           0.692447


### Localization performance at varying CAM thresholds

In [ ]:
path_basedir = '../train_log'